# Correlating Returns

In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Load API Keys from Environment Variables

In [2]:
# Load .env enviroment variables
load_dotenv()

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

## Get AAPL Returns for Past Month

In [3]:
# Set the ticker
ticker = "AAPL"

# Set timeframe to '1D'
timeframe = "1D"

# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp("2020-07-29", tz="America/New_York").isoformat()
past_date = pd.Timestamp("2020-06-29", tz="America/New_York").isoformat()

# Get 4 weeks worth of historical data for AAPL
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=past_date,
    end=current_date,
    after=None,
    until=None,
).df

# Display data
df.head()

AAPL                                    
                             open      high     low   close    volume
2020-06-29 00:00:00-04:00  353.25  362.1736  351.28  362.00  29154717
2020-06-30 00:00:00-04:00  360.08  365.9800  360.00  364.60  28884259
2020-07-01 00:00:00-04:00  365.12  367.3600  363.91  364.01  25429807
2020-07-02 00:00:00-04:00  367.85  370.4700  363.65  364.12  25332853
2020-07-06 00:00:00-04:00  370.00  375.7800  369.87  373.81  26084791

In [4]:
# Drop Outer Table Level
df = df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
df = df.drop(columns=["open", "high", "low", "volume"])

# Since this is daily data, we can keep only the date (remove the time) component of the data
df.index = df.index.date

# Display sample data
df.head()

,close
2020-06-29,362.00
2020-06-30,364.60
2020-07-01,364.01
2020-07-02,364.12
2020-07-06,373.81


In [5]:
# Use the `pct_change` function to calculate daily returns of AAPL
aapl_returns = df.pct_change().dropna()

# Display sample data
aapl_returns.head()

,close
2020-06-30,0.007182
2020-07-01,-0.001618
2020-07-02,0.000302
2020-07-06,0.026612
2020-07-07,-0.003130


In [6]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date),
            to=str(date),
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

In [7]:
# Get first topic
aapl_headlines, dates = get_headlines("apple")

Fetching news about 'apple'
******************************
retrieving news from: 2020-07-29 00:00:00
retrieving news from: 2020-07-28 00:00:00
retrieving news from: 2020-07-27 00:00:00
retrieving news from: 2020-07-26 00:00:00
retrieving news from: 2020-07-25 00:00:00
retrieving news from: 2020-07-24 00:00:00
retrieving news from: 2020-07-23 00:00:00
retrieving news from: 2020-07-22 00:00:00
retrieving news from: 2020-07-21 00:00:00
retrieving news from: 2020-07-20 00:00:00
retrieving news from: 2020-07-19 00:00:00
retrieving news from: 2020-07-18 00:00:00
retrieving news from: 2020-07-17 00:00:00
retrieving news from: 2020-07-16 00:00:00
retrieving news from: 2020-07-15 00:00:00
retrieving news from: 2020-07-14 00:00:00
retrieving news from: 2020-07-13 00:00:00
retrieving news from: 2020-07-12 00:00:00
retrieving news from: 2020-07-11 00:00:00
retrieving news from: 2020-07-10 00:00:00
retrieving news from: 2020-07-09 00:00:00
retrieving news from: 2020-07-08 00:00:00
retrieving news f

In [8]:
# Get second topic
trade_headlines, _ = get_headlines("trade")

Fetching news about 'trade'
******************************
retrieving news from: 2020-07-29 00:00:00
retrieving news from: 2020-07-28 00:00:00
retrieving news from: 2020-07-27 00:00:00
retrieving news from: 2020-07-26 00:00:00
retrieving news from: 2020-07-25 00:00:00
retrieving news from: 2020-07-24 00:00:00
retrieving news from: 2020-07-23 00:00:00
retrieving news from: 2020-07-22 00:00:00
retrieving news from: 2020-07-21 00:00:00
retrieving news from: 2020-07-20 00:00:00
retrieving news from: 2020-07-19 00:00:00
retrieving news from: 2020-07-18 00:00:00
retrieving news from: 2020-07-17 00:00:00
retrieving news from: 2020-07-16 00:00:00
retrieving news from: 2020-07-15 00:00:00
retrieving news from: 2020-07-14 00:00:00
retrieving news from: 2020-07-13 00:00:00
retrieving news from: 2020-07-12 00:00:00
retrieving news from: 2020-07-11 00:00:00
retrieving news from: 2020-07-10 00:00:00
retrieving news from: 2020-07-09 00:00:00
retrieving news from: 2020-07-08 00:00:00
retrieving news f

In [9]:
# Get third topic
economy_headlines, _ = get_headlines("economy")

Fetching news about 'economy'
******************************
retrieving news from: 2020-07-29 00:00:00
retrieving news from: 2020-07-28 00:00:00
retrieving news from: 2020-07-27 00:00:00
retrieving news from: 2020-07-26 00:00:00
retrieving news from: 2020-07-25 00:00:00
retrieving news from: 2020-07-24 00:00:00
retrieving news from: 2020-07-23 00:00:00
retrieving news from: 2020-07-22 00:00:00
retrieving news from: 2020-07-21 00:00:00
retrieving news from: 2020-07-20 00:00:00
retrieving news from: 2020-07-19 00:00:00
retrieving news from: 2020-07-18 00:00:00
retrieving news from: 2020-07-17 00:00:00
retrieving news from: 2020-07-16 00:00:00
retrieving news from: 2020-07-15 00:00:00
retrieving news from: 2020-07-14 00:00:00
retrieving news from: 2020-07-13 00:00:00
retrieving news from: 2020-07-12 00:00:00
retrieving news from: 2020-07-11 00:00:00
retrieving news from: 2020-07-10 00:00:00
retrieving news from: 2020-07-09 00:00:00
retrieving news from: 2020-07-08 00:00:00
retrieving news

In [10]:
# Get fourth topic
iphone_headlines, _ = get_headlines("iphone")

Fetching news about 'iphone'
******************************
retrieving news from: 2020-07-29 00:00:00
retrieving news from: 2020-07-28 00:00:00
retrieving news from: 2020-07-27 00:00:00
retrieving news from: 2020-07-26 00:00:00
retrieving news from: 2020-07-25 00:00:00
retrieving news from: 2020-07-24 00:00:00
retrieving news from: 2020-07-23 00:00:00
retrieving news from: 2020-07-22 00:00:00
retrieving news from: 2020-07-21 00:00:00
retrieving news from: 2020-07-20 00:00:00
retrieving news from: 2020-07-19 00:00:00
retrieving news from: 2020-07-18 00:00:00
retrieving news from: 2020-07-17 00:00:00
retrieving news from: 2020-07-16 00:00:00
retrieving news from: 2020-07-15 00:00:00
retrieving news from: 2020-07-14 00:00:00
retrieving news from: 2020-07-13 00:00:00
retrieving news from: 2020-07-12 00:00:00
retrieving news from: 2020-07-11 00:00:00
retrieving news from: 2020-07-10 00:00:00
retrieving news from: 2020-07-09 00:00:00
retrieving news from: 2020-07-08 00:00:00
retrieving news 

In [11]:
# Get fifth topic
gold_headlines, _ = get_headlines("gold")

Fetching news about 'gold'
******************************
retrieving news from: 2020-07-29 00:00:00
retrieving news from: 2020-07-28 00:00:00
retrieving news from: 2020-07-27 00:00:00
retrieving news from: 2020-07-26 00:00:00
retrieving news from: 2020-07-25 00:00:00
retrieving news from: 2020-07-24 00:00:00
retrieving news from: 2020-07-23 00:00:00
retrieving news from: 2020-07-22 00:00:00
retrieving news from: 2020-07-21 00:00:00
retrieving news from: 2020-07-20 00:00:00
retrieving news from: 2020-07-19 00:00:00
retrieving news from: 2020-07-18 00:00:00
retrieving news from: 2020-07-17 00:00:00
retrieving news from: 2020-07-16 00:00:00
retrieving news from: 2020-07-15 00:00:00
retrieving news from: 2020-07-14 00:00:00
retrieving news from: 2020-07-13 00:00:00
retrieving news from: 2020-07-12 00:00:00
retrieving news from: 2020-07-11 00:00:00
retrieving news from: 2020-07-10 00:00:00
retrieving news from: 2020-07-09 00:00:00
retrieving news from: 2020-07-08 00:00:00
retrieving news fr

In [12]:
# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [13]:
# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [14]:
# Get averages of each topics sentiment
aapl_avg = headline_sentiment_summarizer_avg(aapl_headlines)
trade_avg = headline_sentiment_summarizer_avg(trade_headlines)
economy_avg = headline_sentiment_summarizer_avg(economy_headlines)
iphone_avg = headline_sentiment_summarizer_avg(iphone_headlines)
gold_avg = headline_sentiment_summarizer_avg(gold_headlines)

In [15]:
# Combine Sentiment Averages into DataFrame
topic_sentiments = pd.DataFrame(
    {
        "aapl_avg": aapl_avg,
        "trade_avg": trade_avg,
        "economy_avg": economy_avg,
        "iphone_avg": iphone_avg,
        "gold_avg": gold_avg,
    }
)

In [16]:
# Set the index value of the sentiment averages DataFrame to be the series of dates.
topic_sentiments.index = pd.to_datetime(dates)

In [17]:
# Merge with AAPL returns
topic_sentiments = aapl_returns.join(topic_sentiments).dropna(how="any")

# Display data
display(topic_sentiments)

,close,aapl_avg,trade_avg,economy_avg,iphone_avg,gold_avg
2020-06-30,0.007182,0.179495,-0.161670,-0.068255,0.067280,0.194890
2020-07-01,-0.001618,-0.049925,-0.081785,0.014300,-0.012810,-0.134745
2020-07-02,0.000302,-0.008150,-0.109245,0.126485,0.024030,0.117600
2020-07-06,0.026612,0.184975,0.077800,0.111830,0.027175,-0.038690
2020-07-07,-0.003130,0.096675,-0.137575,0.043045,-0.016675,0.038535
2020-07-08,0.023615,0.090600,-0.022730,-0.078405,0.055520,-0.000975
2020-07-09,0.003198,0.148425,-0.154820,-0.043795,0.142360,0.062310
2020-07-10,0.002248,0.026675,-0.036470,0.084990,0.046870,0.013720
2020-07-13,-0.004198,0.027525,-0.002125,-0.051500,0.054520,0.090415
2020-07-14,0.016496,0.030830,-0.128520,0.008175,0.090475,-0.160830


In [18]:
# Correlate the headlines' sentiment to returns
topic_sentiments.corr().style.background_gradient()

,close,aapl_avg,trade_avg,economy_avg,iphone_avg,gold_avg
close,1.000000,0.236663,-0.172633,-0.387420,0.000473,-0.231050
aapl_avg,0.236663,1.000000,-0.086865,-0.073049,0.087252,0.086046
trade_avg,-0.172633,-0.086865,1.000000,0.253825,0.069636,-0.199496
economy_avg,-0.387420,-0.073049,0.253825,1.000000,-0.254344,0.398676
iphone_avg,0.000473,0.087252,0.069636,-0.254344,1.000000,-0.152079
gold_avg,-0.231050,0.086046,-0.199496,0.398676,-0.152079,1.000000
